<a href="https://colab.research.google.com/github/Khanhnvtb/Text-to-image/blob/main/training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/Khanhnvtb/Text-to-image

fatal: destination path 'Text-to-image' already exists and is not an empty directory.


In [3]:
%cd Text-to-image

/content/Text-to-image


In [4]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.layers import Conv2D, LeakyReLU
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

# Đường dẫn đến thư mục chứa dữ liệu hình ảnh của động vật
data_dir = 'Celebrity Faces Dataset'

# Đặt kích thước hình ảnh mong muốn
image_size = (32, 32)
image_shape = (32, 32, 1)  # Kích thước hình ảnh xám
batch_size = 16  # Giảm batch_size

# Tạo ImageDataGenerator để tải dữ liệu hình ảnh và chuyển đổi
data_generator = ImageDataGenerator(
    rescale=1.0/255,  # Chuẩn hóa dữ liệu hình ảnh về [0, 1]
    validation_split=0.2  # Phân chia dữ liệu thành train set và validation set
)

# Tạo dataset train từ thư mục dữ liệu
train_dataset = data_generator.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',  # Chỉ sử dụng ảnh xám
    class_mode='input',  # Đây là mô hình sinh ảnh, không cần class
    subset='training'  # Sử dụng phần dữ liệu train
)

# Tạo dataset validation từ thư mục dữ liệu
validation_dataset = data_generator.flow_from_directory(
    data_dir,
    target_size=image_size,
    batch_size=batch_size,
    color_mode='grayscale',  # Chỉ sử dụng ảnh xám
    class_mode='input',
    subset='validation'  # Sử dụng phần dữ liệu validation
)

# Xây dựng Generator
def build_generator(latent_dim):
    model = Sequential()
    model.add(Dense(64), input_dim=latent_dim, activation='relu'))
    model.add(Dense(np.prod(image_shape), activation='tanh'))  # Sử dụng tanh để đảm bảo đầu ra trong khoảng [-1, 1]
    model.add(Reshape(image_shape))
    return model

# Xây dựng Discriminator
def build_discriminator():
    model = Sequential()
    model.add(Conv2D(32, (3, 3), strides=(2, 2), padding='same', input_shape=image_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Flatten())
    model.add(Dense(1, activation='sigmoid'))
    return model

# Định nghĩa hàm loss cho Generator và Discriminator
def generator_loss(fake_output):
    return tf.losses.binary_crossentropy(tf.ones_like(fake_output), fake_output)

def discriminator_loss(real_output, fake_output):
    real_loss = tf.losses.binary_crossentropy(tf.ones_like(real_output), real_output)
    fake_loss = tf.losses.binary_crossentropy(tf.zeros_like(fake_output), fake_output)
    return real_loss + fake_loss

# Khởi tạo mô hình và optimizer
latent_dim = 100
generator = build_generator(latent_dim)
discriminator = build_discriminator()

generator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)
discriminator_optimizer = Adam(learning_rate=0.0002, beta_1=0.5)

# Định nghĩa training loop
num_epochs = 50
noise_dim = 50
num_examples_to_generate = 16

# Định nghĩa hàm training step
@tf.function
def train_step(images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)

        real_output = discriminator(images, training=True)
        fake_output = discriminator(generated_images, training=True)

        gen_loss = generator_loss(fake_output)
        disc_loss = discriminator_loss(real_output, fake_output)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss, discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator, discriminator.trainable_variables))


buffer_size = train_dataset.__len__()
dataset = tf.data.Dataset.from_tensor_slices(train_dataset).shuffle(buffer_size).batch(batch_size)

# Hàm để tạo hình ảnh từ mẫu ngẫu nhiên
def generate_and_save_images(model, epoch, test_input):
    predictions = model(test_input, training=False)
    fig = plt.figure(figsize=(4, 4))
    for i in range(predictions.shape[0]):
        plt.subplot(4, 4, i+1)
        plt.imshow(predictions[i, :, :, 0] * 127.5 + 127.5, cmap='gray')
        plt.axis('off')
    plt.savefig(f'image_at_epoch_{epoch:04d}.png')
    plt.show()

# Đào tạo mô hình GAN
def train(dataset, epochs):
    for epoch in range(epochs):
        for image_batch in dataset:
            train_step(image_batch)

        if (epoch + 1) % 10 == 0:
            generate_and_save_images(generator, epoch + 1, tf.random.normal([num_examples_to_generate, noise_dim]))

# Bắt đầu đào tạo
train(train_dataset, num_epochs)


Found 331 images belonging to 17 classes.
Found 75 images belonging to 17 classes.


ValueError: ignored

In [5]:
!git commit -m "update training"

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@2ab112bd5e00.(none)')


In [6]:
!git config --global user.email "khanhtb1012@gmail.com"
!git config --global user.name "Khanhnvtb"

In [7]:
!git commit -m "update training"

On branch main
Your branch is up to date with 'origin/main'.

nothing to commit, working tree clean


In [8]:
!git push origin main

fatal: could not read Username for 'https://github.com': No such device or address


In [9]:
!git add traning.ipynb

fatal: pathspec 'traning.ipynb' did not match any files
